In [1]:
from stop_words import get_stop_words

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and']

In [5]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark Count words") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [152]:
# Testdata
file = spark.sparkContext.wholeTextFiles("data/test/*/*.txt")
filesByLang = file.map(lambda x: (x[0].split("/")[-2], x[1])).reduceByKey(lambda v1,v2: v1+ " " + v2)
seqOp = (lambda x, y: (x[0], y[0]))
combOp = (lambda x, y:(x, y))
occurencesPerLanguage = filesByLang.flatMapValues(lambda l: re.findall("\w+", l))\
        .map(lambda x: (x[0], x[1].lower()))\
        .filter(lambda x: (x[1] not in get_stop_words(x[0].lower())))\
        .map(lambda w: ((w[0], w[1]), 1)).reduceByKey(lambda v1,v2: v1+v2)\
        .map(lambda x: ((x[0][0]), x[0][1], x[1]))\
        .sortBy(lambda a: a[2], ascending=False)\
        .map(lambda x: (x[0], (x[1], x[2])))\
        .groupByKey()


occurencesPerLanguage.mapValues(list).collect()

[('German', [('test', 3), ('hallo', 2), ('neuer', 1), ('tolle', 1)]),
 ('English', [('hey', 3), ('test', 1), ('never', 1), ('know', 1)])]

In [179]:
# real data
file = spark.sparkContext.wholeTextFiles("data/texts/*/*.txt")
filesByLang = file.map(lambda x: (x[0].split("/")[-2], x[1])).reduceByKey(lambda v1,v2: v1+ " " + v2)
occurencesPerLanguage = filesByLang.flatMapValues(lambda l: re.findall("\w+", l))\
        .map(lambda x: (x[0], x[1].lower()))\
        .filter(lambda x: (x[1] not in get_stop_words(x[0].lower())))\
        .map(lambda w: ((w[0], w[1]), 1)).reduceByKey(lambda v1,v2: v1+v2)\
        .map(lambda x: ((x[0][0]), x[0][1], x[1]))\
        .sortBy(lambda a: a[2], ascending=False)\
        .map(lambda x: (x[0], (x[1], x[2])))\
        .groupByKey()


occurences = occurencesPerLanguage.mapValues(list).collect()

In [180]:
for languages in occurences:
    print(languages[0] + str(languages[1][:10]))
    
# TODO: Why we get so many 's' words?

English[('s', 25820), ('said', 18368), ('one', 17930), ('will', 12828), ('upon', 12763), ('now', 11810), ('t', 11782), ('man', 11763), ('time', 9585), ('well', 9482)]
German[('sagte', 10700), ('ja', 8849), ('schon', 7948), ('mehr', 7026), ('wurde', 6489), ('ganz', 6165), ('s', 5967), ('sah', 5858), ('rief', 5683), ('wohl', 5647)]
Russian[('ним', 460), ('глаза', 298), ('друг', 273), ('сердце', 271), ('моей', 255), ('ночь', 241), ('жизни', 234), ('свой', 231), ('знаю', 224), ('горский', 224)]
French[('s', 38328), ('plus', 26737), ('c', 22806), ('dit', 22433), ('bien', 18336), ('homme', 10266), ('dont', 7398), ('rien', 7322), ('puis', 7193), ('monsieur', 7084)]
Ukrainian[('а', 14064), ('й', 11190), ('i', 4832), ('н', 1936), ('о', 1821), ('вiн', 1533), ('бо', 1511), ('се', 1469), ('од', 1421), ('от', 1274)]
Italian[('d', 13788), ('disse', 8759), ('egli', 8691), ('poi', 7288), ('quel', 7199), ('quando', 6762), ('due', 6461), ('s', 6387), ('così', 6239), ('cosa', 5989)]
Spanish[('si', 5444),